# Tokenization med sprogmodeller

Fordelen ved at bruge sprogmodeller er, at metoder til at tokenize er indbygget. Derudover kan vi nemt få lemma af vores tokens, da sprogmodellen netop kender sproget, og derfor kan gradbøje ordene i teksten.

Sprogmodeller hjælper os også til at finde de mere meningsfulde ord. Visse ordklasser (part-of-speech) er ofte "støj" fra et tekstanalyse-perspektiv; fx hjælpeord (AUX), stedord (PRON), forholdsord (ADP) og bindeord (CONJ).

Med en sprogmodel kan vi blot udvælge de ordtyper, som vi vil beholde.

I det nedenstående vises en alternativ måde at tokenize med brug af en dansk sprogmodel fra `spacy`. Denne indeholder desuden en prædefineret stopordsliste, som vi tager i brug.

Da der er tale om twitterdata, frasorteres ord, der starter med "@". spaCy behandler hashtags (#-tegnet) som et token, så derfor er vi nødt til at fortælle spromodellen, at den skal behandle hashtags samlet (# sammen med ordet).

In [3]:
import pandas as pd
import spacy
import re

tweetdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/poltweets_sample.csv"
tweets_df = pd.read_csv(tweetdata_url)

tweet = tweets_df.loc[584, 'full_text']

In [4]:
# Definer tokenizer
nlp = spacy.load("da_core_news_sm") # Indlæser sprogmodel

# Ændring af tokenizer
re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match) # Indlæs regex fra sprogmodel
re_token_match = f"({re_token_match}|#\\w+)" # tilføj hashtag-pattern
nlp.tokenizer.token_match = re.compile(re_token_match).match # opdater tokenizer

custom_stops = ["god", "al", "stor", "ny", "tak", "dag"] # Definerer kontekstspecifikke stopord
default_stopwords = list(nlp.Defaults.stop_words) # Indlæser prædefineret stopordsliste
stop_words = default_stopwords + custom_stops # Danner samlet stopordsliste
pos_tags = ['PROPN', 'ADJ', 'NOUN'] # Definerer POS-tags som skal bevares: egenavne, adjektiver og navneord

doc = nlp(tweet) # Kører sprogmodel på tweet

tokens = [] # Tom liste til at fylde tokens i 

for word in doc: # Looper igennem hvert ord i tweet
    if word.lemma_.startswith("@"): # Ord må ikke starte med @ - går videre til næste ord, hvis det gør
        continue 
    if word.lemma_.startswith("#"): # Ord må ikke starte med # - går videre til næste ord, hvis det gør
        continue 
    if (len(word.lemma_) < 3): # Ord må ikke være mindre end 3 karakterer - går videre til næste ord, hvis det er
        continue
    if (word.pos_ in pos_tags) and (word.lemma_ not in stop_words): # Tjek at ordets POS-tag indgår i listen af accepterede tags og at ordet ikke er stopord
        # Bemærk at denne betingelse nås kun for dem, som ikke opfylder betingelserne fra de andre if-linjer
        tokens.append(word.lemma_) # Tilføj ordets lemma til tokens, hvis if-betingelse er opfyldt

print(tweet) # Print oprindelige tweet
print("\n")            
print(tokens) # Print tokens

Hjemmeværnet, Politikadetterne og Forsvaret klarer ærterne ved grænserne, så Politiet kan kaste sig over at gøre danskere mere trygge. Sikker på,at @ClausOxfeldt bliver lykkelig for vort krav. Han har jo brugt megen energi på at give DFs grænseindsats skylden for mandskabsmangel. https://t.co/PgenOdeSwn


['Hjemmeværnet', 'Politikadetterne', 'Forsvaret', 'ært', 'grænse', 'Politiet', 'dansker', 'tryg', 'Sikker', 'lykkelig', 'krav', 'megen', 'energi', 'DFs', 'grænseindsats', 'skyld', 'mandskabsmangel', 'https://t.co/PgenOdeSwn']


### Tokenizer funktion med spaCy

Ovenstående kan med fordel laves til en funktion, så vi nemt kan foretage samme tokenization på alle tweets i datasættet.

Samtidig genindlæses sprogmodellen, hvor vi slår visse processer fra ("ner"). Dette gør behandlingen hurtigere, da den ellers kører igennem alle processer i sprogmodellen, uanset om disse bruges eller ej.

In [5]:
nlp = spacy.load("da_core_news_sm", disable = ["ner"])

# Ændring af tokenizer
re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match) # Indlæs regex fra sprogmodel
re_token_match = f"({re_token_match}|#\\w+)" # tilføj hashtag-pattern
nlp.tokenizer.token_match = re.compile(re_token_match).match # opdater tokenizer

def tokenizer_spacy(text): # Definerer funktion ud fra koden fra tidligere
    custom_stops = ["god", "al", "stor", "ny", "tak", "dag"] # Definerer kontekstspecifikke stopord
    default_stopwords = list(nlp.Defaults.stop_words) # Indlæser prædefineret stopordsliste
    stop_words = default_stopwords + custom_stops # Danner samlet stopordsliste
    pos_tags = ['PROPN', 'ADJ', 'NOUN'] # Definerer POS-tags som skal bevares: egenavne, adjektiver og navneord

    doc = nlp(text)

    tokens = []

    for word in doc: # Looper igennem hvert ord i tweet
        if word.lemma_.startswith("@"): # Ord må ikke starte med @ - går videre til næste ord, hvis det gør
            continue 
        if word.lemma_.startswith("#"): # Ord må ikke starte med # - går videre til næste ord, hvis det gør
            continue 
        if (len(word.lemma_) < 3): # Ord må ikke være mindre end 3 karakterer - går videre til næste ord, hvis det er
            continue
        if (word.pos_ in pos_tags) and (word.lemma_ not in stop_words): # Tjek at ordets POS-tag indgår i listen af accepterede tags og at ordet ikke er stopord
            # Bemærk at denne betingelse nås kun for dem, som ikke opfylder betingelserne fra de andre if-linjer
            tokens.append(word.lemma_) # Tilføj ordets lemma til tokens, hvis if-betingelse er opfyldt
                
    return(tokens)

Funktionen kan nu bruges på tweets (eller andre strings):

In [6]:
tokens = tokenizer_spacy(tweet)

print(tweet)
print("\n")
print(tokens)

Hjemmeværnet, Politikadetterne og Forsvaret klarer ærterne ved grænserne, så Politiet kan kaste sig over at gøre danskere mere trygge. Sikker på,at @ClausOxfeldt bliver lykkelig for vort krav. Han har jo brugt megen energi på at give DFs grænseindsats skylden for mandskabsmangel. https://t.co/PgenOdeSwn


['Hjemmeværnet', 'Politikadetterne', 'Forsvaret', 'ært', 'grænse', 'Politiet', 'dansker', 'tryg', 'Sikker', 'lykkelig', 'krav', 'megen', 'energi', 'DFs', 'grænseindsats', 'skyld', 'mandskabsmangel', 'https://t.co/PgenOdeSwn']


Tokens kan nemt optælles ved at konvertere dem til en pandas series og bruge `.value_counts`:

In [7]:
tokens_series = pd.Series(tokens)
tokens_series.value_counts()

Hjemmeværnet               1
Politikadetterne           1
mandskabsmangel            1
skyld                      1
grænseindsats              1
DFs                        1
energi                     1
megen                      1
krav                       1
lykkelig                   1
Sikker                     1
tryg                       1
dansker                    1
Politiet                   1
grænse                     1
ært                        1
Forsvaret                  1
https://t.co/PgenOdeSwn    1
dtype: int64